In [ ]:
import tensorflow as tf

In [8]:
def conv2d(input, weight_shape, bias_shape):
    # Calculate the number of input neurons in the weight tensor
    i_n = weight_shape[0] * weight_shape[1] * weight_shape[2]
    
    # Initialize the weights using random values from a normal distribution
    weight_init = tf.random_normal_initializer(stddev=(2.0 / i_n) ** 0.5)
    
    # Create a TensorFlow variable for the weight tensor with the specified shape
    # and initialize it using the weight_init initializer
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    # Initialize the bias term with a constant value of zero
    bias_init = tf.constant_initializer(value=0)
    
    # Create a TensorFlow variable for the bias term with the specified shape
    # and initialize it using the bias_init initializer
    b = tf.get_variable("b", bias_shape, initializer=bias_init)
    
    # Perform a 2D convolution on the input tensor using the weight tensor (W)
    # with a stride of [1, 1, 1, 1] (one step at a time in all directions)
    # and with padding='SAME' to ensure the output has the same dimensions as the input
    conv_out = tf.nn.conv2d(input, W, strides=[1, 1, 1, 1], padding='SAME')
    
    # Add the bias term (b) to the output of the convolutional operation (conv_out)
    # and apply the ReLU activation function to introduce non-linearity
    return tf.nn.relu(tf.nn.bias_add(conv_out, b))



In [9]:
def max_pool(input, k=2):
    # Apply max-pooling operation on the input tensor 'input'.
    # Max-pooling reduces the spatial dimensions of the input tensor
    # by selecting the maximum value from each pooling region.
    
    # 'k' is the size of the max-pooling window (pool size) in both height and width.
    # By default, it's set to 2x2, but you can change it by providing a different value.
    # For example, k=2 means a 2x2 max-pooling window.

    # tf.nn.max_pool performs the max-pooling operation with the specified parameters.
    # The ksize argument defines the size of the pooling window for each dimension.
    # It's set to [1, k, k, 1], where the first and last elements are always 1 (standard values)
    # and the middle elements specify the pooling window size in height and width.

    # The strides argument determines the step size of the pooling window in each dimension.
    # It's set to [1, k, k, 1], meaning that the pooling window moves 'k' units at a time
    # in both height and width directions.

    # The padding argument is set to 'SAME', which means the output feature map will have
    # the same spatial dimensions as the input by adding zero-padding at the edges if needed.

    return tf.nn.max_pool(input, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


In [6]:
def layer(input, weight_shape, bias_shape):
    # Calculate the standard deviation for weight initialization using the given weight_shape.
    # The standard deviation is computed as (2.0 / number of input neurons) ** 0.5.
    weight_stddev = (2.0 / weight_shape[0]) ** 0.5
    
    # Initialize the weights using random values from a normal distribution with the calculated standard deviation.
    w_init = tf.random_normal_initializer(stddev=weight_stddev)
    
    # Initialize the bias term with a constant value of zero.
    bias_init = tf.constant_initializer(value=0)
    
    # Create a TensorFlow variable for the weight tensor with the specified shape,
    # and initialize it using the w_init initializer.
    W = tf.get_variable("W", weight_shape, initializer=w_init)
    
    # Create a TensorFlow variable for the bias term with the specified shape,
    # and initialize it using the bias_init initializer.
    b = tf.get_variable("b", bias_shape, initializer=bias_init)
    
    # Perform a fully connected layer operation (matrix multiplication) on the input tensor using the weight tensor (W),
    # add the bias term (b), and apply the ReLU activation function to introduce non-linearity.
    return tf.nn.relu(tf.matmul(input, W) + b)


In [7]:
def inference(x, keep_prob):
    # Reshape the input tensor 'x' to have shape [-1, 28, 28, 1].
    # '-1' in the first dimension allows the function to handle variable batch sizes.
    # The input tensor represents grayscale images of size 28x28 pixels.
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    # First Convolutional Layer
    with tf.variable_scope("conv_1"):
        # Perform a 2D convolution on the input tensor 'x' using 32 filters of size 5x5.
        # The input has a single channel (grayscale) and the filters produce 32 output channels.
        conv_1 = conv2d(x, [5, 5, 1, 32], [32])
        
        # Apply max-pooling to the output of the first convolutional layer ('conv_1').
        # Reduce the spatial dimensions by selecting the maximum value from each 2x2 region.
        pool_1 = max_pool(conv_1)
    
    # Second Convolutional Layer
    with tf.variable_scope("conv_2"):
        # Perform a 2D convolution on the output of the first pooling layer ('pool_1').
        # Use 64 filters of size 5x5 to process the 32-channel input and produce 64 output channels.
        conv_2 = conv2d(pool_1, [5, 5, 32, 64], [64])
        
        # Apply max-pooling to the output of the second convolutional layer ('conv_2').
        # Reduce the spatial dimensions further using 2x2 max-pooling.
        pool_2 = max_pool(conv_2)
    
    # Fully Connected Layer
    with tf.variable_scope("fc"):
        # Flatten the output of the second pooling layer ('pool_2') into a 1D tensor.
        # The flattened tensor has shape [-1, 7 * 7 * 64], where '-1' represents the batch size.
        pool_2_flat = tf.reshape(pool_2, [-1, 7 * 7 * 64])
        
        # Perform a fully connected layer operation on the flattened tensor ('pool_2_flat').
        # Use 1024 neurons in this fully connected layer.
        fc_1 = layer(pool_2_flat, [7 * 7 * 64, 1024], [1024])
        
        # Apply dropout to the output of the fully connected layer ('fc_1') with a probability of 'keep_prob'.
        # Dropout helps prevent overfitting during training by randomly dropping some neurons during each step.
        fc_1_drop = tf.nn.dropout(fc_1, keep_prob)
    
    # Output Layer
    with tf.variable_scope("output"):
        # Perform another fully connected layer operation on the dropout output ('fc_1_drop').
        # Use 10 neurons in this fully connected layer to produce the final output of the network.
        output = layer(fc_1_drop, [1024, 10], [10])
    
    # Return the output tensor, which represents the predictions of the neural network.
    return output
